# DAX Cheat Sheet for Hackathons

**Reference Guide for Power BI, Excel Power Pivot, and Analysis Services.**

This notebook compiles the most essential Data Analysis Expressions (DAX) patterns based on the latest documentation. It covers aggregations, iterators, time intelligence, and complex filtering logic.

## 1. Aggregations & Iterators (SUM vs SUMX)
Standard aggregations (`SUM`, `AVERAGE`) work on a single column. Iterators (`SUMX`, `AVERAGEX`) iterate through a table row-by-row to evaluate an expression.

In [ ]:
DEFINE
    -- Basic Sum: Aggregates a single column directly
    MEASURE Sales[Total Quantity] = SUM ( Sales[Quantity] )

    -- Iterator (SUMX): Iterates row-by-row. Required when multiplying columns.
    -- 1. Takes the Sales table.
    -- 2. For every row, multiplies Quantity by Net Price.
    -- 3. Sums the results.
    MEASURE Sales[Total Sales Amount] = 
        SUMX ( Sales, Sales[Quantity] * Sales[Net Price] )

    -- Average Iterator (AVERAGEX)
    MEASURE Sales[Avg Line Amount] =
        AVERAGEX ( Sales, Sales[Quantity] * Sales[Net Price] )

EVALUATE
SUMMARIZECOLUMNS (
    'Product'[Color],
    "Total Quantity", [Total Quantity],
    "Total Sales Amount", [Total Sales Amount],
    "Avg Line Amount", [Avg Line Amount]
)

## 2. The Engine: CALCULATE and FILTER
`CALCULATE` is the most powerful function in DAX. It modifies the filter context. `FILTER` is used to create specific table subsets to feed into Calculate.

In [ ]:
EVALUATE
{
    ( 
        "Red Sales", 
        -- CALCULATE changes context. Here it forces Color to be Red.
        CALCULATE ( 
            [Sales Amount], 
            'Product'[Color] = "Red" 
        )
    ),
    ( 
        "High Value Sales",
        -- FILTER is an iterator. It scans the 'Sales' table and keeps rows where Amount > 100.
        -- Using FILTER allows for complex logic not possible in simple boolean statements.
        CALCULATE (
            [Sales Amount],
            FILTER ( Sales, Sales[Quantity] * Sales[Net Price] > 100 )
        )
    )
}

## 3. Handling Filters: ALL, ALLEXCEPT, REMOVEFILTERS
Crucial for calculating percentages (e.g., "Sales % of Total").

In [ ]:
DEFINE 
    MEASURE Sales[All Sales] = 
        -- ALL ignores filters on the specified table/column.
        -- Returns sales for ALL products, regardless of visual selection.
        CALCULATE ( [Sales Amount], ALL ( 'Product' ) )

    MEASURE Sales[% of Total] = 
        DIVIDE ( [Sales Amount], [All Sales] )

    MEASURE Sales[Remove Filters Example] =
        -- REMOVEFILTERS is a semantic alias for ALL when used as a CALCULATE modifier.
        CALCULATE ( [Sales Amount], REMOVEFILTERS ( 'Product' ) )

EVALUATE
SUMMARIZECOLUMNS(
    'Product'[Category],
    "Sales", [Sales Amount],
    "All Sales", [All Sales],
    "% of Total", [Sales[% of Total]]
)

## 4. Time Intelligence: YTD, QTD, and Previous Period
Standard patterns for calculating Year-To-Date and comparing dates. **Note:** Requires a dedicated Date table marked as such in the model.

In [ ]:
DEFINE
    -- TOTALYTD: Syntax sugar for CALCULATE(..., DATESYTD(...))
    MEASURE Sales[Sales YTD] = 
        TOTALYTD ( [Sales Amount], 'Date'[Date] )

    -- Manual YTD Construction (Equivalent to above)
    MEASURE Sales[Sales YTD Manual] = 
        CALCULATE ( 
            [Sales Amount], 
            DATESYTD ( 'Date'[Date] ) 
        )
    
    -- Previous Month: Calculates sales for the month prior to the current context
    MEASURE Sales[Prev Month Sales] =
        CALCULATE (
            [Sales Amount],
            PREVIOUSMONTH ( 'Date'[Date] )
        )

    -- Opening Balance: Evaluates expression at the first date of the period
    MEASURE Sales[Opening Balance Qtr] =
        OPENINGBALANCEQUARTER ( [Sales YTD], 'Date'[Date] )

EVALUATE
    SUMMARIZECOLUMNS (
        'Date'[Calendar Year],
        'Date'[Month],
        "Sales YTD", [Sales YTD],
        "Prev Month Sales", [Prev Month Sales]
    )

## 5. Logic and Conditionals (IF, SWITCH, COALESCE)
Handling branching logic and null values.

In [ ]:
EVALUATE
ADDCOLUMNS (
    VALUES ( 'Product'[Brand] ),
    "Sales Status", 
        VAR CurrentSales = [Sales Amount]
        RETURN
            -- IF: Basic conditional
            IF ( 
                CurrentSales > 3000000, 
                "High Performer", 
                "Standard" 
            ),
    "Brand Tier",
        -- SWITCH: Cleaner than nested IFs. TRUE() allows evaluating conditions.
        SWITCH ( 
            TRUE(),
            [Sales Amount] > 5000000, "Tier 1",
            [Sales Amount] > 1000000, "Tier 2",
            "Tier 3" -- Default else
        ),
    "Safe Sales",
        -- COALESCE: Returns the first non-BLANK argument. Good for handling nulls.
        COALESCE ( [Sales Amount], 0 )
)

## 6. Table Manipulation (SUMMARIZE, ADDCOLUMNS, TOPN)
Functions that return tables rather than scalar values. Essential for virtual tables inside variables.

In [ ]:
EVALUATE
-- TOPN: Returns the top 3 products by Sales Amount
VAR TopProducts = 
    TOPN ( 
        3, 
        VALUES ( 'Product'[Product Name] ), 
        [Sales Amount], 
        DESC 
    )

-- ADDCOLUMNS: Best practice for adding calculations to a summary table
RETURN
    ADDCOLUMNS (
        TopProducts,
        "Current Sales", [Sales Amount],
        "Rank", RANKX ( VALUES('Product'[Product Name]), [Sales Amount] )
    )

## 7. Useful String & Search Functions
Handling text data, concatenation, and searching.

In [ ]:
EVALUATE
ADDCOLUMNS (
    TOPN(5, VALUES('Product'[Product Name])),
    -- SEARCH: Finds position of text. Returns BLANK if not found (via 4th arg).
    "Is Red?", 
        IF ( 
            SEARCH ( "Red", 'Product'[Product Name], 1, 0 ) > 0, 
            "Yes", 
            "No" 
        ),
    -- CONCATENATEX: Combines values from a list into a single string.
    "Colors in Category",
        CONCATENATEX (
            VALUES ( 'Product'[Color] ),
            'Product'[Color],
            ", " -- Separator
        )
)

## 8. Variable Best Practices
Using `VAR` improves readability and performance (evaluates once, reused).

In [ ]:
DEFINE MEASURE Sales[Growth %] = 
    -- Define Variables first
    VAR CurrentSales = [Sales Amount]
    VAR PreviousSales = CALCULATE ( [Sales Amount], SAMEPERIODLASTYEAR ( 'Date'[Date] ) )
    
    -- Return result using variables
    RETURN
        DIVIDE ( 
            CurrentSales - PreviousSales, 
            PreviousSales, 
            0 -- Alt Result on Div/0
        )

EVALUATE
    SUMMARIZECOLUMNS(
        'Date'[Calendar Year],
        "Growth", [Growth %]
    )